![CH5-ADS.png](.\Media\CH5-ADS.png)
# <span style="color:#cc5500;">DBCC CHECKDB Internals and Performance</span>

In order for you to gain a more advanced knowledge of working with and correcting SQL Server corruptions, also involves knowing a bit more about how DBCC CHECKDB works and how to enhance its performance.  If you jumped right to this notebook and you have not already mastered the basics of handling corruption, it is recommended that you go through the previous notebooks in this chapter first.

This notebook will cover:

- Various internals of DBCC CHECKDB
- Trace flags to aid DBCC CHECKDB performance
- Parallelism and memory settings for DBCC CHECKDB

## <span style="color:#cc5500;">What exactly does DBCC CHECKDB do?</span>

- The first thing it does is perform Primitive checks of critical system tables
    - The 3 critical system Storage Engine metadata tables it checkes are sys.allocation\_units, sys.sysrowsets and sys.sysrscols
    - Makes sure that the leaf-level clustered index pages of these 3 metadata tables can be read into memory and looks for any improper loops in the linkage
- Allocation checks.  This is the equivalent of DBCC CHECKALLOC
    - Validates the GAM, SGAM, PFS and IAM Pages bit settings are correct and that no two pages have an identical bit set
- Logical checks of critical system tables
- Logical checks of all other tables in the database
- Metadata checks.  This is the Equivalent to DBCC CHECKCATALOG
- Service Broker data validation
- Indexed view, XML index, spatial index checks
    - Only when the WITH EXTENDED\_LOGICAL\_CHECKS option is used, from SQL Server 2008 onward
- If a repair option is specified, repairs are done at each stage if necessary and possible
- Uses the Query Processor extensively

Fact Processing

- DBCC CHECKDB reads all allocated pages in the database in a very efficient manner
- It reads the pages in allocation order, driving its own readahead, and generates “facts” about what it has seen, which is O(n\*log(n))
    - E.g. facts about pages, b-tree linkages, off-row text linkages
    - Driven by an internal structure called the MultiObjectScanner
- Each fact has a multi-part key
    - Including object ID, index ID, allocation unit ID, page ID
- Facts are passed to the Query Processor for efficient sorting and hashing, then passed back to DBCC for aggregation
    - Aggregation means that facts cancel each other out
    - When extra or missing facts are detected, that’s a corruption
      
        
        ![CH5-FactProcessing.jpg](.\Media\CH5-FactProcessing.jpg)
        ![CH5-FactProcessing2.jpg](.\Media\CH5-FactProcessing2.jpg)
        

Batch Processing

- The fact-storage worktable usually requires more memory than is available so it spills into tempdb
- The batch size is limited to prevent excessive tempdbspace usage
    - A batch’s “size” is the number of tables and associated indexes
- The minimum batch size is a single table and its nonclustered indexes as everything about a table must be checked at the same time
- Tables are added to the batch until one of the following occurs:
    - The total number of indexes in the batch becomes more than 512
    - The estimate for all necessary facts for the batch becomes more than 32 MB
    - If a repair option is specified, the batch size is always limited to a single table
- Read and write performance of tempdb can greatly affect the run time

Parallelism

- DBCC CHECKDB will run in parallel on Enterprise Edition
    
    - Also applies to DBCC CHECKTABLE and DBCC CHECKFILEGROUP
    - DBCC CHECKALLOCand DBCC CHECKCATALOG are always single-threaded
- Each batch executes the DBCC CHECKDB internal “query” and the Query Processor decides how far to parallelize
    
    - Up to the limit imposed by the sp\_configure MAXDOP setting or the Resource Governor workload group MAX\_DOP setting
    - There is no option to set DBCC CHECKDB’s degree of parallelism
- Each thread is given a page to generate facts for, and then calls into the MultiObjectScanner to get the next page to process
    
- Threads then participate in fact aggregation once all pages have been read and processed for facts
    
- Parallelism can be disabled using documented trace flag 2528
    
    - Greatly increases run time, but greatly decreases the resources required
    
    ![CH5-Parallelism.jpg](.\Media\CH5-Parallelism.jpg)

## <span style="color:#cc5500;">Making DBCC CHECKDB go Faster</span>

- Work has been done to improve the performance of DBCC CHECKDB by:
    - Allowing it to check all tables as a single batch
    - Slightly changing the I/O pattern it generates
    - Reducing contention on the DBCC\_MULTIOBJECT\_SCANNER latch
- KB article describing these is at [KB2634571 - Improvements for the DBCC CHECKDB command may result in faster performance when you use the PHYSICAL\_ONLY option (microsoft.com)](https://support.microsoft.com/en-us/topic/kb2634571-improvements-for-the-dbcc-checkdb-command-may-result-in-faster-performance-when-you-use-the-physical-only-option-e49ca6f4-8a5a-7a09-40d3-266ca513177f)
- Ported to SQL Server 2008 R2 and later
    - Uses documented trace flags 2549 and 2562
    - Check the above KB article to see which trace flags are required in which versions
    - Be aware these changes may push the I/O subsystem much harder, and you may not see any changes at all
- Consider reducing DOP to lower run time when using the option WITH PHYSICAL\_ONLY
- On systems with high numbers of physical cores (32+), consider reducing DOP to 4 or 8 to lower run time even without using PHYSICAL\_ONLY

- Consider reducing the memory available to DBCC CHECKDB
- The Query Processor by default will grant DBCC CHECKDB a very large query execution memory grant, potentially causing buffer pool shrinkage
    - Requested grant is many terabytes because of unknown cardinality
    - On a system with 100 GB of memory, grant may be as high as 10 GB
- Performance testing by Jonathan Kehayias has shown that limiting the amount of memory for DBCC CHECKDB to 1 GB can lead to 5-10% performance increase, and very limited buffer pool pressure
- Use Resource Governor to do this
- Jonathan’s blog post at [DBCC CHECKDB Execution Memory Grants - Not Quite What You Expect - Jonathan Kehayias (sqlskills.com)](https://www.sqlskills.com/blogs/jonathan/dbcc-checkdb-execution-memory-grants-not-quite-what-you-expect/)

- Indexes on computed columns drastically slow down DBCC CHECKDB
    - Also applies to DBCC CHECKTABLEand DBCC CHECKFILEGROUP
- Any time a table has an index on a computed column, all threads bottleneck on the DBCC\_OBJECT\_METADATAlatch
    - This controls access to an Expression Evaluator construct in the Query Processor that evaluates computed column expressions
    - The latch can only be acquired in EX mode, hence the bottleneck
- This can lead to 20x performance decrease
- Solution is to disable such indexes while DBCC CHECKDB is running